# Train Fourth Model: NaiveBayes

In [1]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=sD-4n7sPoUt3jmhW1KzrRm3whHC6edOag0zrkSgKhZc&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [reddit-master].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
!gcloud config set project reddit-master

Updated property [core/project].


In [3]:
import pandas as pd
import numpy as np
import pickle as pkl
import nltk.data
import joblib
import nltk
import ast
import logging


from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import datetime
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giuliagalli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
def loggg(msg):
    print("[INFO] {}: {}".format(datetime.datetime.now(), msg))

In [12]:
subreddits = ['aww', 'nba', 'movies', 'todayilearned', 'IAmA', 'Fitness', 'worldnews', 'technology', 'europe', 'politics', 'atheism','science', 'funny', 'gaming']

In [13]:
!ls

07.1_modelling_LRC_TFIDF.ipynb
07.2_modelling_SVM_TFIDF.ipynb
07.3_modelling_RandomForestClassifier_TFIDF.ipynb
07.4_modelling_NB_TFIDF.ipynb
Untitled.ipynb
comments_posts_Fitness.pkl
comments_posts_IAmA.pkl
comments_posts_atheism.pkl
comments_posts_aww.pkl
comments_posts_europe.pkl
comments_posts_funny.pkl
comments_posts_gaming.pkl
comments_posts_movies.pkl
comments_posts_nba.pkl
comments_posts_politics.pkl
comments_posts_science.pkl
comments_posts_technology.pkl
comments_posts_todayilearned.pkl
comments_posts_worldnews.pkl
model_lr.joblib
model_svm.joblib
summary.txt


In [14]:
%%time

model_NB_df = pd.DataFrame(columns = ["subreddit", "body", "subreddit_id"])

for subreddit in subreddits:
    #!gsutil cp gs://reddit_final_results/comments_posts_{subreddit}.pkl .
    loggg("comments_posts_" + subreddit + ".pkl downloaded")

    df = pd.read_pickle("comments_posts_" + subreddit + ".pkl")

    model_NB_df = pd.concat([model_NB_df, df], ignore_index=True)

[INFO] 2019-11-27 15:25:24.616980: comments_posts_aww.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


[INFO] 2019-11-27 15:25:30.131682: comments_posts_nba.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


[INFO] 2019-11-27 15:25:38.925713: comments_posts_movies.pkl downloaded


...

[INFO] 2019-11-27 15:30:31.266462: comments_posts_funny.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


[INFO] 2019-11-27 15:34:37.386303: comments_posts_gaming.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


CPU times: user 2min 19s, sys: 4min, total: 6min 19s
Wall time: 9min 36s


In [15]:
loggg(model_NB_df.shape)

[INFO] 2019-11-27 15:35:01.316251: (10833350, 4)


In [16]:
X = model_NB_df['body']
y = model_NB_df['subreddit_id']

# Definint a fucntion that slit the dataset into three subsets: train and test
def train_dev_test_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    return dict(X_train=X_train,
                X_test=X_test,
                y_train=y_train.astype('int'),
                y_test=y_test.astype('int'))

In [17]:
%time

model_NB_df_split = train_dev_test_split(X,y)

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 15 µs


In [18]:
model_NB_df_split['y_train'].unique()

array([ 0,  1, 13, 10,  3, 12,  5,  7,  8,  4,  2,  9,  6, 11])

In [ ]:
# Multinomial Naive Bayes
#classifier: nb
#parameters:
#    tfidf__max_df: !!python/tuple [0.25, 0.5, 0.75]
#    tfidf__ngram_range: 
#        - !!python/tuple [1, 1]
#        - !!python/tuple [1, 2]
#        - !!python/tuple [1, 3]
#    clf__estimator__alpha: !!python/tuple [0.01, 0.001]


In [ ]:
%%time

model_NB = Pipeline([
    ('tfidf', TfidfVectorizer(preprocessor=" ".join, ngram_range=(1,2))),
    ('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None)))
    ])


model_NB.fit(model_NB_df_split['X_train'], model_NB_df_split['y_train'])

joblib.dump(model_NB, "model_nb.joblib")

In [ ]:
!/Users/giuliagalli/google-cloud-sdk/bin/gsutil cp model_nb.joblib gs://reddit_models/

In [ ]:
prediction = model_lr.predict(model_NB_df_split['X_test'])

loggg("prediction done")

In [ ]:
loggg("reporting Logistic Regression results")

print(classification_report(model_NB_df_split['y_test'], prediction))